# Fitting of TCV q-Profiles

This notebook performs polynomial and piecewise linear fits of the safety factor (q) profiles for TCV plasmas, and compares the resulting fits for q-profile, magnetic shear, and shift. The workflow includes visualization and extraction of fit coefficients for further use.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from src.piecewise_qprofile_generator_inc import Context, run_qprofile_workflow, compare_qprofs

## Resolution and num domain

In [ ]:
Nx = 30
Npieces = 16
x_in = 0.04
x_out = 0.08

## Positive triangularity configuration

In [ ]:
def q_PT_rpRaxis(R, ctx):
    y = R
    a, b, c, d = 497.3420166252413, -1408.736172826569, 1331.4134861681464, -419.00692601227627
    return (a*y**3 + b*y**2 + c*y + d)

ctx_PT = Context(
    a_shift=0.25,
    Z_axis=0.1414361745,
    R_axis=0.8727315068,
    B_axis=1.4,
    R_LCFSmid=1.0968432365089495,
    qfunc=q_PT_rpRaxis,
    x_inner=x_in,
    x_outer=x_out,
    Nx=Nx,
    Npieces=Npieces,
    delta = 0.35
)
PT_qprof_data = run_qprofile_workflow(ctx_PT, q_PT_rpRaxis, return_data=True)

### Profile analysis

In [ ]:
run_qprofile_workflow(ctx_PT, q_PT_rpRaxis, plot=True)

### C Code for piecewise linear representation

In [ ]:
run_qprofile_workflow(ctx_PT, q_PT_rpRaxis, print_code=True)

## Negative triangularity configuration

In [ ]:
def q_NT_rpRaxis(R, ctx):
    y = R
    a, b, c, d = 484.0615913225881, -1378.25993228584, 1309.3099150729233, -414.13270311478726
    return (a*y**3 + b*y**2 + c*y + d)

ctx_NT = Context(
    a_shift=1.0,
    Z_axis=0.1414361745,
    R_axis=0.8867856264,
    B_axis=1.4,
    R_LCFSmid=1.0870056099999,
    qfunc=q_NT_rpRaxis,
    x_inner=x_in,
    x_outer=x_out,
    Nx=Nx,
    Npieces=Npieces,
    delta = -0.38
)

NT_qprof_data = run_qprofile_workflow(ctx_NT, q_NT_rpRaxis, return_data=True)

In [ ]:
run_qprofile_workflow(ctx_NT, q_NT_rpRaxis, plot=True)

### C Code for piecewise linear representation

In [ ]:
run_qprofile_workflow(ctx_NT, q_NT_rpRaxis, print_code=True)

## Comparison

In [ ]:
compare_qprofs(PT_qprof_data, NT_qprof_data)

# Taking into account the Shafranov shift into the q-profile

In [ ]:
ctx_PT_qshaf = ctx_PT
def q_PT_Rshaf(R, ctx): return q_PT_rpRaxis(R,ctx)
ctx_PT_qshaf.Rfunc = 'shaf'
ctx_PT_qshaf.qfunc = q_PT_Rshaf
PT_qshaf_data = run_qprofile_workflow(ctx_PT_qshaf, q_PT_Rshaf, return_data=True, plot=True, print_code=True)
compare_qprofs(PT_qprof_data, PT_qshaf_data)

In [ ]:
# Taking into account the Shafranov shift into the q-profile
ctx_NT_qshaf = ctx_NT
def q_NT_Rshaf(R, ctx): return q_NT_rpRaxis(R,ctx)
ctx_NT_qshaf.Rfunc = 'shaf'
ctx_NT_qshaf.qfunc = q_NT_Rshaf
NT_qshaf_data = run_qprofile_workflow(ctx_NT_qshaf, q_NT_Rshaf, return_data=True, plot=True, print_code=True)
compare_qprofs(NT_qprof_data, NT_qshaf_data)

# Use of a fitted Miller equilibrium
Here we used a least-squares fit to get the Miller parameters and a cubic polynomial to fit the q-profile from rho=0.5 to rho=1.0.

In [ ]:
def q_PT_cubfit(R, ctx):
    y = R
    a, b, c, d = 1848.7402338072004, -5603.130135833314, 5667.704706413577, -1912.2515772834936
    return (a*y**3 + b*y**2 + c*y + d)

ctx_PT_milopt = Context(
    a_shift=0.408025,
    delta = 0.282550,
    R_axis=0.872732,
    Z_axis=0.141436,
    R_LCFSmid=1.096856,
    B_axis=1.4,
    qfunc=q_PT_cubfit,
    x_inner=x_in,
    x_outer=x_out,
    Nx=Nx,
    Npieces=Npieces,
    Rfunc='shaf'
)

PT_milopt = run_qprofile_workflow(ctx_PT_milopt, q_PT_cubfit, return_data=True, plot=True, print_code=True)
compare_qprofs(PT_milopt, PT_qshaf_data)

In [ ]:
(ctx_PT_milopt.R_LCFSmid - ctx_PT_milopt.R_axis) / ctx_PT_milopt.a_mid()

In [ ]:
def q_NT_cubfit(R, ctx):
    y = R
    a, b, c, d = 1599.1288806286452, -4829.173575118684, 4867.160132081546, -1636.104964605296
    return (a*y**3 + b*y**2 + c*y + d)

# ctx_NT_milopt = ctx_NT_qshaf
# ctx_NT_milopt.a_shift = 1.0000
# ctx_NT_milopt.R_LCFSmid = 1.087473
# ctx_NT_milopt.Z_axis = 0.138879
# ctx_NT_milopt.R_axis = 0.886786
# ctx_NT_milopt.delta = -0.259153
# ctx_NT_milopt.qfunc = q_NT_cubfit

ctx_NT_milopt = Context(
    a_shift=1.0000,
    Z_axis=0.138879,
    R_axis=0.886786,
    B_axis=1.4,
    R_LCFSmid=1.087473,
    qfunc=q_NT_cubfit,
    x_inner=x_in,
    x_outer=x_out,
    Nx=Nx,
    Npieces=Npieces,
    delta = -0.259153,
    Rfunc='shaf'
)

NT_milopt = run_qprofile_workflow(ctx_NT_milopt, q_NT_cubfit, return_data=True, plot=True, print_code=True)
compare_qprofs(NT_milopt, NT_qshaf_data)

In [ ]:
compare_qprofs(PT_milopt, NT_milopt)